
# ⋆⭒˚.⋆USPTO Dataset Preprocessing and Neo4j Import with Noctis 🌙⋆⭒˚.⋆

This Jupyter notebook is your go-to resource for navigating the complete workflow for the USPTO dataset. It walks you through converting the data into a format ready for graph ingestion using Noctis and importing it into a Neo4j database. You'll find examples of querying the database with built-in queries, along with guidance on crafting and executing your own custom queries, all designed to help you efficiently explore and interact with the patent data in a graph-based setup.


## Table of Contents
0️⃣ 📥 Downloading & unpacking the USPTO dataset

1️⃣ 🏗️ Defining a graph schema with Noctis

2️⃣ 📦 Bulk data ingestion (CSV) into Neo4j

3️⃣ 🔄 Incremental update into Neo4j

4️⃣ 🔍 Querying the graph database

## 0. 📥 Download USPTO dataset
This is a preparatory step to load the data for this example exercise



In [ ]:
import os
import urllib.request
import zipfile
import pandas as pd

# Link to USPTO dataset

DATA_URL = 'https://github.com/wengong-jin/nips17-rexgen/blob/master/USPTO/data.zip?raw=true'

# These are folders where the data will be downloaded and processed
# Adjust to your liking

RAW_DIR = './data/raw'
PROCESSED_DIR = './data/processed'
os.makedirs(RAW_DIR, exist_ok=True)
os.makedirs(PROCESSED_DIR, exist_ok=True)

# File names which are going to be used throughout this notebook 

USPTO_FILE = 'uspto.csv'

# First five lines of USPTO for quick checks

FIVELINES_FILE = 'uspto_5lines.csv'

NOCTIS_HEADER = ['ChemicalEquation.smiles', 'ChemicalEquation.reaction_center']

In [ ]:
# Download and extract

zip_path = os.path.join(RAW_DIR, 'uspto_data.zip')
urllib.request.urlretrieve(DATA_URL, zip_path)
with zipfile.ZipFile(zip_path, 'r') as z:
    z.extractall(RAW_DIR)

# Combine splits

files = ['train.txt', 'test.txt', 'valid.txt']
RAWDATA_DIR = os.path.join(RAW_DIR, 'data')

combined = pd.concat(
    [pd.read_csv(os.path.join(RAWDATA_DIR, f), sep=' ', header=None, names=NOCTIS_HEADER) for f in files],
    ignore_index=True
)
combined.to_csv(os.path.join(PROCESSED_DIR, USPTO_FILE), index=False)

In [ ]:
# Read the first 5 lines of the CSV file

df = pd.read_csv(os.path.join(PROCESSED_DIR, USPTO_FILE), delimiter=',', nrows=5)

# Write the first 5 lines to a new CSV file

df.to_csv(os.path.join(PROCESSED_DIR, FIVELINES_FILE), sep=',', lineterminator='\n', index=False)

## 1. 🏗️ Defining a graph schema with Noctis
Define node & relationship types either from a dictionary, YAML or JSON file.

In [ ]:
from noctis.data_architecture.graph_schema import GraphSchema

In [ ]:
gs = GraphSchema.build_from_dict({'extra_nodes':{'molecule':'ExtraMolecule', 'chemical_equation':'ExtraChemicalEquation'}})

#gs = GraphSchema.build_from_file(file_path = 'schema.yaml', file_format = 'yaml')

In [ ]:
## The graph schema can aslo be saved to YAML or JSON

gs.save_to_file('schema_saved.yaml', file_format='yaml')

In [ ]:
print(gs)

## 2. 📦 Bulk data ingestion (CSV) into Neo4j

### A) Preprocessing
How to prepare large volumes of data for Neo4j ingestion

In [ ]:
from noctis.data_transformation.preprocessing.data_preprocessing import Preprocessor


In [ ]:
Preprocessor.info()

In [ ]:
# To initialise Preprocessor with default graph schema

preprocessor = Preprocessor() 

# To initialise Preprocessor with custom schema

preprocessor = Preprocessor(schema = gs) 


In [ ]:
# Preprocessor configurations can be loaded from a YAML file 

preprocessor.set_config_from_yaml(file_path = 'preprocessor_config.yaml')

In [ ]:
# To make sure that CSV file is interpreted correctly, follow the suggested header format
# In the case of USPTO, the header looks like this

print(NOCTIS_HEADER)

# which means that each ChemicalEquation node will have a property reaction_center 

In [ ]:
INPUT_FILE = os.path.join(PROCESSED_DIR, FIVELINES_FILE)

# Preprocessing can be done in parallel

preprocessor.preprocess_csv_for_neo4j_parallel(input_file=INPUT_FILE, validation = True, output_folder='../output_uspto', prefix='USPTO',delimiter=',', blocksize=1000 )
# We offer an option to validate reaction strings, to ensure consistency of the data and its correct transformation into a graph 
# You can also turn it off at your own risk..

# Users have the option to provide their own Dask Client instance for parallel processing.
# Example usage: preprocessor.preprocess_csv_for_neo4j_parallel(input_file=INPUT_FILE, dask_client=Client(...))
# Note: If users supply their own Dask Client, they must also manage closing the connection themselves.


In [ ]:
# Alternatively, it's also possible to process files in serial mode:
# preprocessor.preprocess_csv_for_neo4j_serial(input_file='./data/processed/uspto_5lines.csv', output_folder='../output_test', prefix='USPTO',delimiter=',', chunksize=5 )
# where chunksize defines how many rows in one partition the preprocessor is going to handle.
# In serial mode it is also possible to define nrows, which will limit the preprocessing to the first n rows. 
# If chunksize is not defined, the whole file will be processed in one go. 

In [ ]:
# If some reactions fail, or if some rows have empty reaction strings, they can be found in corresponding files for further investigation

In [ ]:
# Both graph schema and configurations can be saved into files, so it can be used to reproduce the preprocessing.

preprocessor.schema.save_to_file(file_path ='schema_saved.yaml', file_format ='yaml')
preprocessor.config.save_to_yaml(file_path = './config_saved.yaml')

### B) Importing
Now that we've preprocessed our data, we're ready to establish a connection with Neo4j DB and import our prepared datasets into the graph database.
<br> We assume that you have already Neo4j DB instance configured and started.


In [ ]:
from noctis import settings

# IMPORTANT: Use secret management system to store your secret information
# such as database URIs, usernames, and password. Never hardcode these values in your scripts.
# Repository by default is instantiated with credentials defined in settings. 
# If you configured your setting, no need to pass the attributes

URI = settings.NEO4J_DEV_URL
USERNAME = settings.NEO4J_DEV_USER # default username in Neo4j is "neo4j". If you didn't define another name explicitly through Neo4j Desktop, your username is "neo4j"
PASSWORD = settings.NEO4J_DEV_PASSWORD

print(URI, USERNAME, PASSWORD)


In [ ]:
from noctis.repository.neo4j.neo4j_repository import Neo4jRepository

# Repository takes as an input GraphSchema object to dynamically define correct node labels and relationships types
# If you want to use default GraphSchema, you don't have to pass it to the repository

# The default database in Neo4j is 'neo4j'
repo = Neo4jRepository(database='uspto', schema = gs)

# With credentials not from settings
# repo = Neo4jRepository(uri='bolt://localhost:0000', username="neo4j", password='123123123', database='uspto')

In [ ]:
# Before importing data, create uniqueness constraints
# They are created automatically for all nodes defined in GraphSchema based on UID

repo.create_constraints()

In [ ]:
repo.show_constraints()

In [ ]:
IMPORT_PATH = <YOUR IMPORT PATH>
# If your Neo4j is set up to have an import directory,
# first move the files to the import directory before running this query and remove folder_path attribute
# If your Neo4j is set up to import files from anywhere in your file system, define the import path

repo.execute_query(query_name = 'import_db_from_csv', folder_path = IMPORT_PATH, prefix = 'USPTO')
# Now your reaction data is graphormed! 

In [ ]:
# You can see that the graph schema in Neo4j GDB is the same as you defined in the beginning

repo.execute_query('get_gdb_schema')

## 3. 🔄 Incremental update into Neo4j

### A) Preprocessing
Reaction data can be added to the graph not only from a CSV file, but also from a Python Object. This is suggested for small volumes of data

In [ ]:
# Let's see how reactions from a list of reaction strings can be added to the GDB

my_list_of_strings = ['N#Cc1ccsc1N.O=[N+]([O-])c1cc(F)c(F)cc1F>>N#Cc1ccsc1Nc1cc(F)c(F)cc1[N+](=O)[O-]','O=Cc1cncc(Cl)c1COC1CCCCO1>>OCc1cncc(Cl)c1COC1CCCCO1', 'NC1CCN(CC2Cn3c(=O)ccc4ncc(F)c2c43)CC1O.O=Cc1cc2c(cn1)OCS2>>Cl.O=c1ccc2ncc(F)c3c2n1CC3CN1CCC(NCc2cc3c(cn2)OCS3)C(O)C1', 'WRONG>>STRING']

In [ ]:
# When data is loaded from a list of reactions, only base nodes and relationships can be generated

preprocessor = Preprocessor()

In [ ]:
# This creates a DataContainer class -- an internal data class of Noctis

reaction_data = preprocessor.preprocess_object_for_neo4j(data = my_list_of_strings , data_type = 'reaction_string')

In [ ]:
# If some of the reactions will not be processed, they can be fetched from preprocessor

preprocessor.get_failed_strings()

### B) Merging
DataContainer object can be merged to the Graph. If generated nodes don't exist in the Graph, they will be created, but if they exist already, they will not be added/duplicated.

In [ ]:
repo.execute_query(query_name = 'load_nodes_and_relationships', data = reaction_data, data_type = 'data_container')


In [ ]:
# Python objects could be loaded also directly through this query, but in this way you won't be able to access failed reaction strings

repo.execute_query(query_name = 'load_nodes_and_relationships', data = my_list_of_strings, data_type = 'reaction_string')

## 4. 🔍 Querying the graph database
All the data is in the graph! It's time to run some queries
<br> We implemented some built-in queries but also left an option to run easily your own queries
<Br> First, about the hard-coded queries:

In [ ]:
# All the available queries and their input parameters are listed in the helper functino of Neo4jRepository class

Neo4jRepository.info()

In [ ]:
# Each query depending on its type can return either an unprocessed Neo4j output (modify_graph), or a pandas DataFrame (retrieve_stats), or a DataContainer (retrieve_graph)
# Let's try running a retrieve_graph type of query

node = repo.execute_query(query_name = 'get_node',node_uid="M100203740101976085840472731949187371946" )

In [ ]:
import yaml

# Users can define their own queries in a yaml, which has a very simple format.
# Each query should have query_name, query_type and query itself
# Cypher query can be juts copy-pasted into the YAML
# Query types are the same as for built-in queries: modify_graph, retrieve_stats, retrieve_graph
# If it requires arguments, it's also possible to define query_args_required and query_args_optional

CUSTOM_QUERY_YAML = './queries/custom_query.yaml'

with open(CUSTOM_QUERY_YAML, 'r') as file:
    yaml_data = yaml.safe_load(file)
    print(yaml.dump(yaml_data, default_flow_style=False))

In [ ]:
# This is how you can execute a custom query

node_by_smiles = repo.execute_custom_query_from_yaml(yaml_file='queries/custom_query.yaml', query_name='get_node_by_smiles', smiles ='COc1ccc(CN(Cc2ccc(OC)cc2)c2nc(C)nc(-c3cc(CN4CCN(S(C)(=O)=O)CC4C)cnc3Nc3cnc(OC)c(F)c3)n2)cc1')
print(node_by_smiles)

In [ ]:
# This custom query retrieve_stats as pandas.DataFrame

repo.execute_custom_query_from_yaml(yaml_file='queries/custom_query.yaml', query_name='count_nodes')

In [ ]:
# The queries which return Noctis DataContainer can be transformed in other python objects
# Let's retrieve some routes for this molecule

routes = repo.execute_query('get_routes', root_node_uid="M100203740101976085840472731949187371946", max_number_reactions=7)

In [ ]:
print(routes)

In [ ]:
# DataContainer info method has a short example of what can be done with DataContainer
# and list of available transformations

routes.info()

In [ ]:
# Noctis DataContainer can be transformed into list of reaction strings, netowrkX objects, or pandas DataFrames
# When transformed to pandas, two data frames are returned:
# One contains Nodes and the other Relationships. 
# The record_id is used to maintain the association between nodes and relationships that are part of a single result record 
# in a query output

routes.transform_to(format_type='pandas')[0]

In [ ]:
routes.transform_to(format_type='pandas')[1]